Time Calibration of the Mu2e Calorimeter

by Giacinto boccia

version 0.1 | 2024-09-05

In [138]:
import numpy as np
import awkward as ak
import uproot
import quantities as pq
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [139]:
#Cut parameters
HITNUM_CUT = 1
Q_MIN_CUT = 4000
Q_MAX_CUT = 8000
COS_THETA_CUT = 0.9
CHI_ON_NDF_CUT = 2000
#constants
N_ROWS = 36
N_COLUMNS = 28
N_SIPMS = 2

In [140]:
#Input
hits_path = input("Hits file to process:")
cal_path = input("Starting caibration file:") or False
n_runs = int(input("Iterations to perform:"))
#Name of the tree inside the file
hits_path += ":sidet"

In [141]:
#Opening the calibration start-point file
if cal_path:
    print("Starting calibration file not yet supported")
    cal_corection = pq.Quantity(np.zeros((N_ROWS, N_COLUMNS, N_SIPMS)), 'ns')
else:
    #The time-correction array is [rows, columns, SiPM] shaped
    cal_corection = pq.Quantity(np.zeros((N_ROWS, N_COLUMNS, N_SIPMS)), 'ns')

In [142]:
#Loading the tree, start by defining custom event class that can compute residurals
class Cosmic(ak.Record):     
    def t_residuals(self) -> dict | None:
        #returns a dictionary with '(row, col, sipm)' as key and the residual as value
        #Filters on the hit charge 
        q_min_fitler = self.Qval > Q_MIN_CUT
        q_max_fitler = self.Qval < Q_MAX_CUT
        q_filter = q_max_fitler & q_min_fitler
        if self.slope:
            #Select events that have a not "None" slope. For each hit return the time residual
            cos_theta = 1 / np.sqrt(1 + self.slope ** 2)
            t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
            t_res = dict()
            for i, [row, col, sipm] in enumerate(zip(self.iRow[q_filter],
                                                     self.iCol[q_filter],
                                                     self.SiPM[q_filter])):
                #Apply current time corrections
                t_arr[i] += cal_corection[row, col, sipm]
            #I can't really understand why this np.average operation is losing the ns, 
            #assigning them again schouldn't be necessary, but it is
            t_0_ev = pq.Quantity(np.average(t_arr, weights= self.Qval[q_filter]), 'ns')
            for time, y, row, col, sipm in zip(t_arr, 
                                               self.Yval[q_filter],
                                               self.iRow[q_filter],
                                               self.iCol[q_filter],
                                               self.SiPM[q_filter]):
                #Each residual is stored in the dictionary with '(row, col, sipm)' as key
                y = y * pq.cm
                residual = time + y / (pq.c * cos_theta) - t_0_ev
                t_res[str(row) + ', ' + str(col) + ', ' + str(sipm)] = residual.item()
            return t_res
        else:
            return None                
ak.behavior["cosmic"] = Cosmic
        
#Loading tree in an array structure, we only need some of the branches
branches = ("nrun", "nsubrun", "evnum", "nHits", "iRow", "iCol", "SiPM", "Xval", "Yval", "Qval", "Tval", "templTime")
with uproot.open(hits_path) as file:
    tree = file.arrays(filter_name = branches, entry_stop= 1000)
    
#Now change the array so that it uses the custom class defned above
tree = ak.Array(tree, with_name= "cosmic")

In [143]:
#To get the parameters, each event is fitted
def linear_fit(event) -> dict[str, np.double | int]:
    q_min_fitler = event.Qval > Q_MIN_CUT
    q_max_fitler = event.Qval < Q_MAX_CUT
    x_arr = event.Xval[q_min_fitler & q_max_fitler]
    y_arr = event.Yval[q_min_fitler & q_max_fitler]
    if len(x_arr) > 1:
        #Events that are not empty
        if np.max(x_arr) - np.min(x_arr) > 34.4:
            #Events that are not vertical
            [slope, intercept], residuals, _, _, _ = np.polyfit(x_arr, y_arr, deg= 1, full= True)
            chi_sq : np.double = np.sum(residuals)
            ndf = event.nHits - 2
            return {'vertical' : False, 'slope' : slope, 'intercept' : intercept, 'chi_sq' : chi_sq, 'ndf' : ndf}
        else:
            #Vertical events get flagged
            return {'vertical': True, 'slope' : None, 'intercept' : None, 'chi_sq' : None, 'ndf' : None}
    else:
        return None
        
with ProcessPoolExecutor() as executor:
    fit_results = list(executor.map(linear_fit, tree))
fit_results = ak.Array(fit_results)

In [144]:
#Add the fit results to the tree
if hasattr(fit_results, 'vertical'):
    tree = ak.with_field(tree, fit_results.vertical, "vertical")
    tree = ak.with_field(tree, fit_results.slope, "slope")
    tree = ak.with_field(tree, fit_results.intercept, "intercept")
    tree = ak.with_field(tree, fit_results.chi_sq, "chi_sq")
    tree = ak.with_field(tree, fit_results.ndf, "ndf")

In [155]:
#Apply the filters and compute the residuals
slope_cut = np.sqrt(1 / COS_THETA_CUT - 1)
filters = {'n_min' : tree.nHits > HITNUM_CUT,
           'slope_min' : abs(tree.slope) > slope_cut,
           'chi_sq' : tree.chi_sq / tree.ndf < CHI_ON_NDF_CUT}
filtered_events = tree[filters['n_min'] & filters['slope_min'] & filters['chi_sq']]
filtered_events = ak.drop_none(filtered_events)

#We don't really need to store the means, unless we want to do a plot
mean_res_arr = np.zeros([n_runs, N_ROWS, N_COLUMNS, N_SIPMS]) * pq.ns
for run_n in range(n_runs):
    #Get the residuals
    def get_t_residuals(event) -> dict | None:
        #Just a wrapper since the executor wants a funciton and not a method
        return event.t_residuals()
    with ProcessPoolExecutor() as executor:
        residuals = list(executor.map(get_t_residuals, filtered_events))
    residuals = ak.Array(residuals)

    def residuals_to_correction(channel : str) -> None:
        chan_address = tuple(int(entry) for entry in channel.split(', '))
        mean_res = np.mean(residuals[channel]) * pq.ns
        mean_res_arr[run_n, chan_address] = mean_res
        cal_corection[chan_address] -= mean_res

    with ThreadPoolExecutor() as executor:
        #This funciton can't be parallelized with ProcessPoolExecutor, but there are a maximum of 674 calls,
        #hopefully, ThreadPoolExecutor expedites it a little
        executor.map(residuals_to_correction, residuals.fields)

/tmp/ipykernel_18796/444587635.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
/tmp/ipykernel_18796/444587635.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
/tmp/ipykernel_18796/444587635.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
/tmp/ipykernel_18796/444587635.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ 

In [168]:
residuals.fields

['6, 25, 1',
 '6, 25, 0',
 '9, 9, 0',
 '22, 9, 0',
 '24, 12, 0',
 '22, 9, 1',
 '24, 12, 1',
 '23, 11, 0',
 '24, 13, 0',
 '23, 11, 1',
 '24, 13, 1',
 '4, 24, 1',
 '8, 10, 1',
 '5, 23, 1',
 '6, 23, 1',
 '4, 24, 0',
 '7, 16, 0',
 '5, 22, 0',
 '8, 10, 0',
 '6, 23, 0',
 '9, 9, 1',
 '22, 10, 0',
 '22, 10, 1',
 '16, 10, 0',
 '15, 9, 0',
 '16, 10, 1',
 '15, 9, 1',
 '24, 11, 0',
 '23, 12, 0',
 '24, 11, 1',
 '23, 12, 1',
 '23, 14, 0',
 '23, 14, 1',
 '5, 24, 1',
 '5, 25, 1',
 '8, 12, 1',
 '5, 24, 0',
 '5, 25, 0',
 '8, 12, 0',
 '15, 12, 0',
 '16, 12, 0',
 '14, 13, 0',
 '16, 12, 1',
 '15, 12, 1',
 '14, 13, 1',
 '4, 23, 0',
 '4, 22, 1',
 '4, 22, 0',
 '5, 23, 0',
 '24, 14, 0',
 '24, 14, 1',
 '16, 7, 0',
 '16, 7, 1',
 '14, 15, 0',
 '16, 11, 0',
 '14, 15, 1',
 '16, 11, 1',
 '22, 12, 0',
 '22, 12, 1',
 '23, 13, 1',
 '7, 18, 1',
 '15, 10, 0',
 '15, 10, 1',
 '23, 15, 0',
 '23, 15, 1',
 '16, 9, 0',
 '16, 9, 1',
 '14, 14, 0',
 '15, 11, 1',
 '8, 11, 1',
 '23, 10, 0',
 '23, 10, 1',
 '23, 9, 1',
 '15, 11, 0',


In [169]:
mean_res_arr[:, 22, 9, 1]

array([-13.04039719, -12.57302334, -12.1582583 , -11.78848189]) * ns

In [170]:
cal_corection[22, 9, 1]

array(29.5105543) * ns